# Automated ML



Import Dependencies.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-138176
aml-quickstarts-138176
southcentralus
d4ad7261-832d-46b2-b093-22156001df5b


In [4]:
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-138176,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "nueva"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview


The dataset was download from Kaggle: https://www.kaggle.com/thegurusteam/spanish-high-speed-rail-system-ticket-pricing
Then I have to create some new variables for example day of the week, delete empty rows and create a sample dataset with fewer obs. The reason why I had to perform a sample is that azure needs more than one hour to train an automl model when there are 30.000.000 obs.
Then I uploaded the dataset:


In [16]:
found = False
key = "datalite"
description_text = "dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        print("error")


df = dataset.to_pandas_dataframe()
df.describe()

,duration,departureDay,departureMonth,ALICANTE,BARCELONA,CADIZ,CASTELLO,CASTELLON,CIUDAD REAL,CORDOBA,...,4,5,6,earlyMorning,lateNight,midday,midmorning,morning,night,y
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,2.914604,16.044400,5.584200,0.005800,0.129800,0.00060,0.000800,0.004200,0.001400,0.027400,...,0.144800,0.110400,0.130800,0.007400,0.182400,0.235200,0.177800,0.197200,0.04700,58.024988
std,1.601784,8.725987,2.273575,0.075944,0.336117,0.02449,0.028276,0.064678,0.037394,0.163262,...,0.351934,0.313419,0.337215,0.085713,0.386212,0.424166,0.382383,0.397924,0.21166,25.032202
min,0.380000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,9.450000
25%,1.870000,9.000000,4.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,38.700000
50%,2.530000,16.000000,5.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,55.700000
75%,3.130000,24.000000,7.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,75.400000
max,11.520000,31.000000,12.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,183.500000


## AutoML Configuration



I have choosen regression as task because I need to predic a continue variable. I choose normalized_root_mean_squared_error as primary metric because it is one of the metric that a regression model try to minimize.

In [89]:


automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'normalized_root_mean_squared_error',
    "task": "regression",
    "training_data":dataset,
    "label_column_name":"y",   
    "path" : project_folder,
    "enable_early_stopping": True,
                             "featurization": 'auto',
                             "debug_log":"automl_errors.log",
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    **automl_settings)

## Run Details

In [65]:
# use the `RunDetails` widget to show the different experiments.

In [90]:
pipeline_run = experiment.submit(automl_config)

Running on remote.


In [91]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values

{'runId': 'AutoML_f2c50d27-8a14-4a62-b507-b44b080b76d8',
 'target': 'nueva',
 'status': 'Completed',
 'startTimeUtc': '2021-02-08T19:56:13.490131Z',
 'endTimeUtc': '2021-02-08T20:24:53.540714Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'normalized_root_mean_squared_error',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'nueva',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-138176","workspace_name":"quick-starts-ws-138176","region":"southcentralus","compute_target":"nueva","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"normalized_root_mean_squared_error","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":nul

## Best Model

In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [92]:
best_run, best_model = pipeline_run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [94]:
print(best_model)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [96]:
print(best_model._final_estimator)

PreFittedSoftVotingRegressor(estimators=[('0',
                                          Pipeline(memory=None,
                                                   steps=[('maxabsscaler',
                                                           MaxAbsScaler(copy=True)),
                                                          ('lightgbmregressor',
                                                           LightGBMRegressor(boosting_type='gbdt',
                                                                             class_weight=None,
                                                                             colsample_bytree=1.0,
                                                                             importance_type='split',
                                                                             learning_rate=0.1,
                                                                             max_depth=-1,
                                                                  

In [97]:
best_model.steps

[('datatransformer',
  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                  feature_sweeping_config=None, feature_sweeping_timeout=None,
                  featurization_config=None, force_text_dnn=None,
                  is_cross_validation=None, is_onnx_compatible=None, logger=None,
                  observer=None, task=None, working_dir=None)),
 ('prefittedsoftvotingregressor',
  PreFittedSoftVotingRegressor(estimators=[('0',
                                            Pipeline(memory=None,
                                                     steps=[('maxabsscaler',
                                                             MaxAbsScaler(copy=True)),
                                                            ('lightgbmregressor',
                                                             LightGBMRegressor(boosting_type='gbdt',
                                                                               class_weight=None,
                             

In [104]:
print(best_run)

Run(Experiment: ml-experiment-1,
Id: AutoML_f2c50d27-8a14-4a62-b507-b44b080b76d8_38,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [111]:
import joblib
joblib.dump(value=best_model, filename='bestautomlmodel.pkl')

['bestautomlmodel.pkl']

In [112]:
from azureml.core.model import Model

In [113]:
model = Model.register(workspace=ws,model_name = "bestautomlmodel",
                       model_path = 'bestautomlmodel.pkl')

Registering model bestautomlmodel


In [124]:
from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'condaEnv.yml')
env = Environment.from_conda_specification('condaEnv', 'condaEnv.yml')

inference_config = InferenceConfig(entry_script='score.py', environment=env)


In [126]:
from azureml.core.webservice import LocalWebservice, AciWebservice
aci_config=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)




aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, 
                                                auth_enabled=True)
service = model.deploy(workspace=ws,name="aml",
                       models=[model],inference_config=inference_config,
                       deployment_config=aci_config)



In [127]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [128]:
service.update(enable_app_insights=True)

In [129]:
print(service.state)

Healthy


In [130]:
print(service.scoring_uri)

http://0d199411-5575-4192-a08c-ef41d701b9e8.southcentralus.azurecontainer.io/score
